In [ ]:
import pandas as pd
import geopandas as gpd
import pysal as ps
import pandana as pdna
import urbanaccess as ua
from urbanaccess.config import settings
from urbanaccess.gtfsfeeds import feeds
from urbanaccess import gtfsfeeds
from urbanaccess.gtfs.gtfsfeeds_dataframe import gtfsfeeds_dfs
from urbanaccess.network import ua_network, load_network
import cartopy
from pathlib import Path

In [ ]:
data_folder = Path('../Data/')

# Intégration du réseau de transports publics avec le réseau piéton

In [ ]:
settings.to_dict()
settings.log_console = True
settings.log_console = False

## Feeds object

In [ ]:
feeds.to_dict()

## Searching for GTFS feeds

In [ ]:
gtfs_path = '/Users/david/Dropbox/PhD/GitHub/SanteIntegra/Data/raw/OFS/Transports publics/gtfs_fp2022_2022-04-13_04-15'

In [ ]:
gtfs_path

In [ ]:
validation = True
verbose = True
# bbox for Geneva
bbox = (5.9559,46.1225,6.3215,46.3758)
bbox_ch = (5.8358140744676303, 45.659168946713827, 10.979311848153316, 47.869910020393519)
remove_stops_outsidebbox = True
append_definitions = True

In [ ]:
loaded_feeds =  ua.gtfs.load.gtfsfeed_to_df(gtfs_path, validation=validation, verbose=verbose, bbox = bbox_ch, remove_stops_outsidebbox=remove_stops_outsidebbox, append_definitions=append_definitions)

In [ ]:
loaded_feeds.stops.head()

In [ ]:
loaded_feeds.stops.unique_agency_id.unique()

In [ ]:
loaded_feeds.stops.plot(kind = 'scatter',x = 'stop_lon',y = 'stop_lat', s=0.1)

In [ ]:
ua.gtfs.network.create_transit_net(gtfsfeeds_dfs=loaded_feeds,
                                   day='monday',
                                   timerange=['07:00:00', '10:00:00'],
                                   calendar_dates_lookup=None)

In [ ]:
urbanaccess_net = ua.network.ua_network

In [ ]:
urbanaccess_net.transit_edges.head()

In [ ]:
urbanaccess_net.transit_nodes.head()

In [ ]:
urbanaccess_net.transit_nodes.plot(kind='scatter', x='x', y='y', s=0.1)

### Pedestrian network

In [ ]:
nodes, edges = ua.osm.load.ua_network_from_bbox(bbox=bbox_ch,
                                                remove_lcn=True)

In [ ]:
ua.osm.network.create_osm_net(osm_edges=edges,
                              osm_nodes=nodes,
                              travel_speed_mph=3)

In [ ]:
# Let's inspect the results which we can access inside of the existing urbanaccess_net variable:
urbanaccess_net.osm_nodes.head()

In [ ]:
urbanaccess_net.osm_edges.head()

In [ ]:
urbanaccess_net.osm_nodes.plot(kind='scatter', x='x', y='y', s=0.1)

### Create an integrated transit and pedestrian network
Now let's integrate the two networks together. The resulting graph will be added to your existing UrbanAccess network object. After running this step, your network will be ready to be used with Pandana.

The resulting integrated network will be added to your UrbanAccess network object inside net_nodes and net_edges

In [ ]:
ua.network.integrate_network(urbanaccess_network=urbanaccess_net,
                             headways=False)

In [ ]:
# Let's inspect the results which we can access inside of the existing urbanaccess_net variable:
urbanaccess_net.net_nodes.head()

In [ ]:
urbanaccess_net.net_edges.head()

In [ ]:
urbanaccess_net.net_edges[urbanaccess_net.net_edges['net_type'] == 'transit'].head()

### Save the network to disk

You can save the final processed integrated network net_nodes and net_edges to disk inside of a HDF5 file. By default the file will be saved to the directory of this notebook in the folder data.

In [ ]:
ua.network.save_network(urbanaccess_network=urbanaccess_net,
                        filename='final_net.h5',
                        overwrite_key = True)

### Load the network

In [ ]:
urbanaccess_net = ua.network.load_network(filename='final_net.h5')

### Visualize the network
You can visualize the network you just created using basic UrbanAccess plot functions

**Integrated network**

In [ ]:
ua.plot.plot_net(nodes=urbanaccess_net.net_nodes,
                 edges=urbanaccess_net.net_edges,
                 bbox=bbox,
                 fig_height=30, margin=0.02,
                 edge_color='#999999', edge_linewidth=1, edge_alpha=1,
                 node_color='black', node_size=1.1, node_alpha=1, node_edgecolor='none', node_zorder=3, nodes_only=False)

**Integrated network by travel time**
Use the col_colors function to color edges by travel time. In this case the darker red the higher the travel times.

Note the ability to see AC Transit's major bus arterial routes (in darker red) and transfer locations and BART rail network (rail stations are visible by the multiple bus connections at certain junctions in the network most visible in downtown Oakland at 19th, 12th Street, and Lake Merritt stations and Fruitvale and Coliseum stations) with the underlying pedestrian network. Downtown Oakland is located near the white cutout in the northeast middle section of the network which represents Lake Merritt.

In [ ]:
edgecolor = ua.plot.col_colors(df=urbanaccess_net.net_edges, col='weight', cmap='gist_heat_r', num_bins=5)
ua.plot.plot_net(nodes=urbanaccess_net.net_nodes,
                 edges=urbanaccess_net.net_edges,
                 bbox=bbox,
                 fig_height=30, margin=0.02,
                 edge_color=edgecolor, edge_linewidth=1, edge_alpha=0.7,
                 node_color='black', node_size=0, node_alpha=1, node_edgecolor='none', node_zorder=3, nodes_only=False)

Let's zoom in closer to the canton of Geneva using a new smaller extent bbox. Note the bus routes on the major arterials and the TPG routes from station to station.

In [ ]:
edgecolor = ua.plot.col_colors(df=urbanaccess_net.net_edges, col='weight', cmap='gist_heat_r', num_bins=5)
ua.plot.plot_net(nodes=urbanaccess_net.net_nodes,
                 edges=urbanaccess_net.net_edges,
                 bbox=bbox,
                 fig_height=30, margin=0.02,
                 edge_color=edgecolor, edge_linewidth=1, edge_alpha=0.7,
                 node_color='black', node_size=0, node_alpha=1, node_edgecolor='none', node_zorder=3, nodes_only=False)

### Transit network

In [ ]:
ua.plot.plot_net(nodes=urbanaccess_net.net_nodes,
                 edges=urbanaccess_net.net_edges[urbanaccess_net.net_edges['net_type']=='transit'],
                 bbox=bbox,
                 fig_height=30, margin=0.02,
                 edge_color='#999999', edge_linewidth=1, edge_alpha=1,
                 node_color='black', node_size=0, node_alpha=1, node_edgecolor='none', node_zorder=3, nodes_only=False)

### Pedestrian network

In [ ]:
ua.plot.plot_net(nodes=urbanaccess_net.net_nodes,
                 edges=urbanaccess_net.net_edges[urbanaccess_net.net_edges['net_type']=='walk'],
                 bbox=None,
                 fig_height=30, margin=0.02,
                 edge_color='#999999', edge_linewidth=1, edge_alpha=1,
                 node_color='black', node_size=0, node_alpha=1, node_edgecolor='none', node_zorder=3, nodes_only=False)

In [ ]:
import time
s_time = time.time()
transit_ped_net = pdna.Network(urbanaccess_net.net_nodes["x"],
                               urbanaccess_net.net_nodes["y"],
                               urbanaccess_net.net_edges["from_int"],
                               urbanaccess_net.net_edges["to_int"],
                               urbanaccess_net.net_edges[["weight"]], 
                               twoway=False)
print('Took {:,.2f} seconds'.format(time.time() - s_time))